### Website traffic analysis

In [15]:
#import wget

#wget.download("https://gist.githubusercontent.com/escape-velocity-labs/17c3ba12aef542afe5055a859e0fbd98/raw/af9e2431fb69c76618aeec9940cd5fd6deab8b1f/style.css"
#              , out="assets")
#wget.download("https://gist.githubusercontent.com/escape-velocity-labs/81b8225d72306541018bf868f3a23436/raw/c14fdac538abc45f8e19c4828bb015aa931c41c5/web_data.csv"
#              , out="data")

'data/web_data.csv'

In [ ]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

### Dashboard structure

#### Dashboard header

#### Evolution of website visits

#### First row

#### Sales funnel

#### Distribution of visits by attribute

#### Second row

#### Visits by date and time

#### Distribution by country

#### Third row

#### Create the dashboard